# Example: Forested Area by Admin Level 2

## Setup

In [1]:
import ee
from earthengine_dask.core import ClusterGEE
import google.auth
import pandas as pd
from pprint import pprint

## Authenticate & Initialize Earth Engine

Get credentials and the GCP project ID, authenticating if necessary.

In [2]:
try:
    credentials, project_id = google.auth.default()
except google.auth.exceptions.DefaultCredentialsError:
    !gcloud auth application-default login
    credentials, project_id = google.auth.default()
try:
    ee.Initialize(credentials=credentials, project=project_id)
except google.auth.exceptions.RefreshError:
    !gcloud auth application-default login
    credentials, project_id = google.auth.default()
ee.Initialize(credentials=credentials, project=project_id)

# Input Data

## Input: Forest Baseline

This example will use the [European Commission Joint Research Centre's 2020 global map of forest cover](https://data.jrc.ec.europa.eu/dataset/10d1b337-b7d1-4938-a048-686c8185b290) for the forest baseline. The dataset is [available in Earth Engine](https://developers.google.com/earth-engine/datasets/catalog/JRC_GFC2020_V1).

In [3]:
ic = ee.ImageCollection("JRC/GFC2020/V1")

In [4]:
pprint(ic.getInfo())

{'bands': [],
 'features': [{'bands': [{'crs': 'EPSG:4326',
                          'crs_transform': [8.983152841195215e-05,
                                            0,
                                            -170.00005897568744,
                                            0,
                                            -8.983152841195215e-05,
                                            80.03737653225383],
                          'data_type': {'max': 255,
                                        'min': 0,
                                        'precision': 'int',
                                        'type': 'PixelType'},
                          'dimensions': [4007503, 1559941],
                          'id': 'Map'}],
               'id': 'JRC/GFC2020/V1/2020',
               'properties': {'system:asset_size': 59186087403,
                              'system:footprint': {'coordinates': [[-180, -90],
                                                                   [1

In [5]:
print(f'There is {ic.size().getInfo()} image in the collection.')

There is 1 image in the collection.


... which we will use as the forest baseline.

In [6]:
jrc_forest_baseline = ic.first()

Verify that the image is binary.

In [7]:
jrc_forest_baseline.unmask().reduceRegion(
    reducer=ee.Reducer.minMax(),
    geometry=ee.Geometry.BBox(10, 10, 11, 11),
    maxPixels=1e10,
).getInfo()

{'Map_max': 1, 'Map_min': 0}

Looking at the projection information, the image is in decimal degrees of latitude and longitude (EPSG:4326).

In [8]:
proj_info = jrc_forest_baseline.projection().getInfo()
pprint(proj_info)

{'crs': 'EPSG:4326',
 'transform': [8.983152841195215e-05,
               0,
               -170.00005897568744,
               0,
               -8.983152841195215e-05,
               80.03737653225383],
 'type': 'Projection'}


In [9]:
print(f'The nominal scale (at the equator) is '
      f'{jrc_forest_baseline.projection().nominalScale().getInfo()} meters/pixel.')

The nominal scale (at the equator) is 10 meters/pixel.


## Input: Administrative Boundaries

### GeoBoundaries ADM2

We will use the municipal level (ADM2) boundaries provided by the [geoBoundaries](https://www.geoboundaries.org/) global database of political administrative boundaries v6.0, which is also [available in Earth Engine](https://developers.google.com/earth-engine/datasets/catalog/WM_geoLab_geoBoundaries_600_ADM2).

In [10]:
geoboundaries_adm2 = ee.FeatureCollection("WM/geoLab/geoBoundaries/600/ADM2")
# geoboundaries_adm2 = geoboundaries_adm2.filter(ee.Filter.eq('shapeGroup', 'CAN'))
# geoboundaries_adm2 = geoboundaries_adm2.filter(ee.Filter.eq('shapeName', 'Colorado'))
# geoboundaries_adm2 = geoboundaries_adm2.filter(ee.Filter.eq('shapeName', 'Boulder'))

# roi = ee.Geometry.Polygon(
#         [[[-109.01952260759319, 40.971552045695994],
#           [-109.01952260759319, 37.01127149086416],
#           [-101.99925893571819, 37.01127149086416],
#           [-101.99925893571819, 40.971552045695994]]], None, False)
# geoboundaries_adm2 = geoboundaries_adm2.filterBounds(roi)

geoboundary_properties = None  # i.e. use all the properties

There are quite a few features in the collection.

In [11]:
print(f'There are {geoboundaries_adm2.size().getInfo()} features in the collection.')

There are 49617 features in the collection.


In [12]:
geoboundaries_adm2.first().getInfo()['properties']

{'shapeGroup': 'VAT',
 'shapeID': '52476719B53484862947726',
 'shapeName': 'Vatican City',
 'shapeType': 'ADM0'}

In [13]:
# admin.aggregate_histogram('shapeName').getInfo()

In [14]:
geoboundaries_id_field = 'shapeID'
geoboundaries_adm2_list = geoboundaries_adm2.aggregate_histogram(geoboundaries_id_field).getInfo().keys()

# Analysis

Define a function that calculates the forested area, and adds it back to the feature.

In [15]:
def get_area(img, admin_fc, id_field, shape_id, properties=None):

    fc = ee.FeatureCollection(
        admin_fc.filter(ee.Filter.eq(id_field, shape_id))
    )
    feat = fc.first()

    stats_sum = ee.Number(
        img.multiply(ee.Image.pixelArea()).reduceRegions(
            collection=fc,
            reducer=ee.Reducer.sum(),
        ).aggregate_array('sum').get(0)
    )
    prop_dict = feat.toDictionary(properties).set('area_km2', stats_sum.round().multiply(1e-6))
    
    return ee.Dictionary(prop_dict).getInfo()

In [16]:
# Try to run one
region_id = list(geoboundaries_adm2_list)[0]  # 'area_km2': 7968.522155
# region_id = '811477B26547766372806'  # problematic shape; Times out after 5 minutes
print(region_id)
test = get_area(
    img=jrc_forest_baseline,
    admin_fc=geoboundaries_adm2,
    id_field=geoboundaries_id_field,
    shape_id=region_id,
    properties=geoboundary_properties
)
test

10022922B10293113814705


{'area_km2': 3274.467658,
 'shapeGroup': 'MDG',
 'shapeID': '10022922B10293113814705',
 'shapeName': 'Ambanja',
 'shapeType': 'ADM2'}

In [17]:
def get_area_v2(
        img,
        feat,
        properties=None,
        max_pixels=1e10,
    ):
    stats_sum_temp = (
        img.multiply(
            ee.Image.pixelArea()
        ).reduceRegion(
            geometry=feat.geometry(),
            reducer=ee.Reducer.sum(),
            maxPixels=max_pixels,
        )
    )    
    results = img.bandNames().map(
        lambda band : ee.Number(stats_sum_temp.get(band)).round().multiply(1e-6)
    ).getInfo()
    # Return results for just the first band.
    prop_dict = feat.toDictionary(properties).set('area_km2', results[0])
    return ee.Dictionary(prop_dict).getInfo()

In [18]:
# Aggreate a section at a time
def get_area_v3(
        img,
        feat,
        properties=None,
        max_pixels=1e11,
        covering_grid_scale=16,
    ):
    feat = ee.Feature(feat)
    
    proj = img.projection().scale(2**covering_grid_scale, 2**covering_grid_scale)

    covering_grid = feat.geometry().coveringGrid(proj)
    print('covering_grid.size() = ', covering_grid.size().getInfo())

    grids = covering_grid.toList(1000)
    geoms = grids.map(lambda f : ee.Feature(f).geometry())
    area_total = 0
    for grid_geom in geoms.getInfo():
        # print('grid_geom', grid_geom)
        f_grid = ee.Feature(grid_geom)
        # TODO: test sensitivity to proj=img.projection(), maxError, reversing feat/f_grid
        feature_piece = f_grid.intersection(feat, maxError=0.01, proj=img.projection())
        # print('feature_piece', feature_piece.getInfo())
        piece_area = get_area_v2(
            img=img,
            feat=feature_piece,
            properties=properties,
            max_pixels=max_pixels,
        )
        print('piece_area', piece_area.get('area_km2'))
        area_total += piece_area.get('area_km2')
    # print('area_total', area_total)
    prop_dict = feat.toDictionary(properties).set('area_km2', area_total)
    return ee.Dictionary(prop_dict).getInfo()


In [29]:
def process_features(
        img, 
        fc,
        covering_grid_scale
    ):
    id_list = fc.aggregate_array('system:index').getInfo()
    for feature_id in id_list:
        feat = fc.filter(ee.Filter.eq('system:index', feature_id)).first()
        print(
            get_area_v3(
                img,
                feat,
                properties=None,
                covering_grid_scale=covering_grid_scale
            )
        )

geoboundaries_adm2 = ee.FeatureCollection("WM/geoLab/geoBoundaries/600/ADM2")
# process_features(img=jrc_forest_baseline, fc=geoboundaries_adm2.filter(ee.Filter.eq('system:index', '00000000000000001c3a')))  # Icatu, Brazil
# process_features(img=jrc_forest_baseline, fc=geoboundaries_adm2.filter(ee.Filter.eq('system:index', '00010000000000001fce')))  # midsize Canada region
process_features(
    img=jrc_forest_baseline,
    fc=geoboundaries_adm2.filter(
        ee.Filter.eq('system:index', '00010000000000001ff3')
    ),
    covering_grid_scale=16,
)  # Very large Northern Canada


covering_grid.size() =  23
piece_area 0
piece_area 0.11370999999999999
piece_area 0
piece_area 0
piece_area 1134.748005
piece_area 4926.709285
piece_area 3071.793705
piece_area 2.459267
piece_area 0.041839999999999995
piece_area 0
piece_area 2723.368936
piece_area 37584.488913
piece_area 80671.838106
piece_area 57368.787766999994
piece_area 32780.717534999996
piece_area 4585.2317809999995
piece_area 159.044887
piece_area 225.228455
piece_area 31331.114333999998
piece_area 57098.651902
piece_area 37785.879343
piece_area 44006.422944
piece_area 10798.106555999999
{'area_km2': 406254.747271, 'shapeGroup': 'CAN', 'shapeID': '811477B98274245708554', 'shapeName': 'Northwest Territories / Terri*', 'shapeType': 'ADM2'}


## Very large Northern Canada (00010000000000001ff3)

```
process_features(
    img=jrc_forest_baseline,
    fc=geoboundaries_adm2.filter(
        ee.Filter.eq('system:index', '00010000000000001ff3')
    ),
    covering_grid_scale=17,
)  # Very large Northern Canada
```
Output (10 min)
```
covering_grid.size() =  8
piece_area 1134.748005
piece_area 7998.50299
piece_area 2.614817
piece_area 0
piece_area 2723.368936
piece_area 149812.669808
piece_area 185034.036546
piece_area 59548.806166999995
{'area_km2': 406254.7472689999, 'shapeGroup': 'CAN', 'shapeID': '811477B98274245708554', 'shapeName': 'Northwest Territories / Terri*', 'shapeType': 'ADM2'}
```

```
process_features(
    img=jrc_forest_baseline,
    fc=geoboundaries_adm2.filter(
        ee.Filter.eq('system:index', '00010000000000001ff3')
    ),
    covering_grid_scale=16,
)  # Very large Northern Canada
```
Output (??? mins)
```
covering_grid.size() =  23
piece_area 0
piece_area 0.11370999999999999
piece_area 0
piece_area 0
piece_area 1134.748005
piece_area 4926.709285
piece_area 3071.793705
piece_area 2.459267
piece_area 0.041839999999999995
piece_area 0
piece_area 2723.368936
piece_area 37584.488913
piece_area 80671.838106
piece_area 57368.787766999994
piece_area 32780.717534999996
piece_area 4585.2317809999995
piece_area 159.044887
piece_area 225.228455
piece_area 31331.114333999998
piece_area 57098.651902
piece_area 37785.879343
piece_area 44006.422944
piece_area 10798.106555999999
{'area_km2': 406254.747271, 'shapeGroup': 'CAN', 'shapeID': '811477B98274245708554', 'shapeName': 'Northwest Territories / Terri*', 'shapeType': 'ADM2'}
```

## Hawaii 4 islands (00020000000000000e0b)
```
process_features(
    img=jrc_forest_baseline,
    fc=geoboundaries_adm2.filter(
        ee.Filter.eq('system:index', '00020000000000000e0b')
    ),
    covering_grid_scale=14,
)  # Hawaii 4 islands
```
Output:
```
covering_grid.size() =  4
piece_area 25.086164999999998
piece_area 0.124369
piece_area 246.526116
piece_area 834.487941
{'area_km2': 1106.224591, 'shapeGroup': 'USA', 'shapeID': '52423323B70973006487009', 'shapeName': 'Maui', 'shapeType': 'ADM2'}
```


```
process_features(
    img=jrc_forest_baseline,
    fc=geoboundaries_adm2.filter(
        ee.Filter.eq('system:index', '00020000000000000e0b')
    ),
    covering_grid_scale=15,
)  # Hawaii 4 islands
```
Output:
```
covering_grid.size() =  2
piece_area 25.210534
piece_area 1081.014056
{'area_km2': 1106.22459, 'shapeGroup': 'USA', 'shapeID': '52423323B70973006487009', 'shapeName': 'Maui', 'shapeType': 'ADM2'}
```

```
process_features(
    img=jrc_forest_baseline,
    fc=geoboundaries_adm2.filter(
        ee.Filter.eq('system:index', '00020000000000000e0b')
    ),
    covering_grid_scale=16,
)  # Hawaii 4 islands
```
Output:
```
covering_grid.size() =  2
piece_area 25.210534
piece_area 1081.014056
{'area_km2': 1106.22459, 'shapeGroup': 'USA', 'shapeID': '52423323B70973006487009', 'shapeName': 'Maui', 'shapeType': 'ADM2'}
```

## larger Canada region test
```
process_features(
    img=jrc_forest_baseline,
    fc=geoboundaries_adm2.filter(
        ee.Filter.eq('system:index', '00010000000000001fd2')
    ),
    covering_grid_scale=17,
)  # larger Canada region
```
Output
```
covering_grid.size() =  1
piece_area 222307.474921
{'area_km2': 222307.474921, 'shapeGroup': 'CAN', 'shapeID': '811477B61209589162148', 'shapeName': 'Northeast / Nord-est', 'shapeType': 'ADM2'}
```

```
process_features(
    img=jrc_forest_baseline,
    fc=geoboundaries_adm2.filter(
        ee.Filter.eq('system:index', '00010000000000001fd2')
    ),
    covering_grid_scale=16,
)  # larger Canada region
```
Output
```
covering_grid.size() =  4
piece_area 17097.961574
piece_area 8281.468789
piece_area 111576.56083799999
piece_area 85351.483719
{'area_km2': 222307.47492, 'shapeGroup': 'CAN', 'shapeID': '811477B61209589162148', 'shapeName': 'Northeast / Nord-est', 'shapeType': 'ADM2'}
```

```
process_features(
    img=jrc_forest_baseline,
    fc=geoboundaries_adm2.filter(
        ee.Filter.eq('system:index', '00010000000000001fd2')
    ),
    covering_grid_scale=15,
)  # larger Canada region
```
output
```
covering_grid.size() =  10
piece_area 8160.096154999999
piece_area 8937.865419
piece_area 8281.468789
piece_area 18003.483688
piece_area 59734.079387
piece_area 41074.933781
piece_area 796.078213
piece_area 33042.919551
piece_area 38216.286307999995
piece_area 6060.2636299999995
{'area_km2': 222307.47492100002, 'shapeGroup': 'CAN', 'shapeID': '811477B61209589162148', 'shapeName': 'Northeast / Nord-est', 'shapeType': 'ADM2'}
```

In [ ]:
## problematic polygon results

## covering_grid_scale = 17 (? min)
# covering_grid.size() =  17

## covering_grid_scale = 16 (30 min)
# covering_grid.size() =  44
# piece_area 0
# piece_area 0
# piece_area 0
# piece_area 0
# piece_area 0
# piece_area 0
# piece_area 0
# piece_area 0
# piece_area 0
# piece_area 0
# piece_area 0
# piece_area 0
# piece_area 0
# piece_area 0
# piece_area 0
# piece_area 0
# piece_area 0.008088
# piece_area 0.027437999999999997
# piece_area 0.098663
# piece_area 1.14146
# piece_area 0.035799
# piece_area 0.178333
# piece_area 0.694444
# piece_area 0.077383
# piece_area 0.005212
# piece_area 20.209909
# piece_area 200.61459
# piece_area 15.432127999999999
# piece_area 46.374755
# piece_area 20.339371
# piece_area 0.6439969999999999
# piece_area 0.9231809999999999
# piece_area 1.8692369999999998
# piece_area 4.3092
# piece_area 0.568255
# piece_area 0.015165999999999999
# piece_area 3888.65222
# piece_area 2585.778612
# piece_area 0.123474
# piece_area 0
# piece_area 0.128498
# piece_area 0.007783
# piece_area 77.314599
# piece_area 540.940532
# {'area_km2': 7406.512327, 'shapeGroup': 'CAN', 'shapeID': '811477B26547766372806', 'shapeName': 'Nunavut', 'shapeType': 'ADM2'}

## covering_grid_scale = 15 (60 min)
# covering_grid.size() =  134
# {'area_km2': 7406.512329000001, 'shapeGroup': 'CAN', 'shapeID': '811477B26547766372806', 'shapeName': 'Nunavut', 'shapeType': 'ADM2'}



## covering_grid_scale = 17 (? min)
# covering_grid.size() =  17

## covering_grid_scale = 16 (30 min)
# covering_grid.size() =  44
# piece_area 0
# piece_area 0
# piece_area 0
# piece_area 0
# piece_area 0
# piece_area 0
# piece_area 0
# piece_area 0
# piece_area 0
# piece_area 0
# piece_area 0
# piece_area 0
# piece_area 0
# piece_area 0
# piece_area 0
# piece_area 0
# piece_area 0.008088
# piece_area 0.027437999999999997
# piece_area 0.098663
# piece_area 1.14146
# piece_area 0.035799
# piece_area 0.178333
# piece_area 0.694444
# piece_area 0.077383
# piece_area 0.005212
# piece_area 20.209909
# piece_area 200.61459
# piece_area 15.432127999999999
# piece_area 46.374755
# piece_area 20.339371
# piece_area 0.6439969999999999
# piece_area 0.9231809999999999
# piece_area 1.8692369999999998
# piece_area 4.3092
# piece_area 0.568255
# piece_area 0.015165999999999999
# piece_area 3888.65222
# piece_area 2585.778612
# piece_area 0.123474
# piece_area 0
# piece_area 0.128498
# piece_area 0.007783
# piece_area 77.314599
# piece_area 540.940532
# {'area_km2': 7406.512327, 'shapeGroup': 'CAN', 'shapeID': '811477B26547766372806', 'shapeName': 'Nunavut', 'shapeType': 'ADM2'}

## covering_grid_scale = 15 (60 min)
# covering_grid.size() =  134
# {'area_km2': 7406.512329000001, 'shapeGroup': 'CAN', 'shapeID': '811477B26547766372806', 'shapeName': 'Nunavut', 'shapeType': 'ADM2'}


In [ ]:
print('med region in Canada')
print(f'v2 {12844.829692}')
print(f'v3 {9017.708524 + 3827.1211359999998}')
print(f'v3 {9017.714211 + 3827.120167} proj=proj')
print(f'v3 {9017.714211 + 3827.120167} proj=img.projection()')
print('feature_piece = feat.intersection(f_grid, maxError=0.01, proj=img.projection())')
print(f'v3 {9017.714071 + 3827.120203}')

print('larger region in Canada')
print(f'v2 {222307.47882699998}')
print(f'v3 {17097.953794 + 8281.468525 + 111576.568655 + 85351.48794199999}')

In [ ]:
# # Use for debugging to count the total area, rather than forested area
# forest_baseline = forest_baseline.unmask().multiply(0).add(1)

In [ ]:
# Try it out.
# region_list = admin.aggregate_array('shapeID').distinct().getInfo()
# region_list

In [ ]:
# # tileScale=1
# get_area(img=forest_baseline, shape_id='42512837B26705409874577')

In [ ]:
# # tileScale=16
# get_area(img=forest_baseline, shape_id='42512837B26705409874577')

## Start Dask Cluster

Start up a Earth Engine enabled cluster. This may take a few minutes to complete.

In [ ]:
cluster = ClusterGEE(
    name='test-cluster-forest-by-admin-temp4',
    n_workers=5,
    worker_cpu=8,
    # spot_policy="spot_with_fallback",
    region='us-west1',
    idle_timeout="4 hours",
)

Retrieve a client for the cluster, and display it.

In [ ]:
client = cluster.get_client()
client

In [ ]:
# Problematic shape?
#region_list = admin.aggregate_array('shapeID').distinct().getInfo()
# region_list = ['42512837B26705409874577']

In [ ]:
# Create and submit jobs among the workers.
# Allow for retries to handle "Too many concurrent aggregations." errors
submitted_jobs = [
    client.submit(
            get_area,  # function
            binary_image, admin_fc, id_field, region_id, properties,  # function parameters
            retries=1
        )
    for region_id in admin_regions
]

In [ ]:
## Debug issue with large regions by trying a problematic shape_id
# submitted_jobs = [
#     {
#         'shape_id': '42512837B26705409874577',
#         'tile_scale': tile_scale,
#         'area':client.submit(
#             get_area, forest_baseline, '42512837B26705409874577',
#             retries=1
#         )
#     }
# ]

In [ ]:
# for job in submitted_jobs:
#     if job.status in ['error']:
#         future = job
#         print(job)
#         print(future.exception())

jobs_with_errors = [
    (job, job.exception(), )
    for job in submitted_jobs
    if job.status in ['error']
]
jobs_with_errors

## Get finished jobs

In [ ]:
finished_jobs = [job for job in submitted_jobs if job.status=='finished']
print(len(finished_jobs))

In [ ]:
finished_results = client.gather(finished_jobs)
finished_results[-5:]

In [ ]:
df = pd.DataFrame(finished_results)
df

In [ ]:
# Show regions with the maximum alert area
df.sort_values('area_km2').tail()

In [ ]:
# Write the results to a CSV file
with open('output.csv', 'w') as out:
    df.to_csv(out)

In [ ]:
#client.shutdown()